# word2vec(avg)+DNN


## 1.导入工具包

In [1]:
import os
import csv
import time
import datetime
import random
import json
from collections import Counter
from math import sqrt

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed,Input,Conv2D,MaxPool2D,concatenate,Flatten,Dense,Dropout,Embedding,SimpleRNN,Reshape,GRU,LSTM
from tensorflow.keras import Sequential,optimizers,losses
from tensorflow.keras.models import Model,model_from_yaml
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from bs4 import BeautifulSoup
import logging
from gensim.corpora import Dictionary
import multiprocessing
import yaml
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import jieba

## 2.设置参数

In [2]:
vocab_dim = 60
maxlen = 50
n_iterations = 1  # ideally more..
n_exposures = 10
window_size = 7
batch_size = 64
n_epoch = 10
input_length = 60
cpu_count = multiprocessing.cpu_count()

## 3.读取数据集

In [3]:
neg=pd.read_excel('../../data/day10-nlp-data/film_review/neg.xlsx',index=None,header = None)
pos=pd.read_excel('../../data/day10-nlp-data/film_review/pos.xlsx',index=None,header = None)

## 4. 查看数据集

### 消极评论

In [4]:
neg.head()

,0,1
0,搞笑不搞笑，荒诞不荒诞，你拍电影还是演话剧，拉这么多人跟你一起弄这么个破电影，耍猴呐？我看过...,0
1,可能是我明白很多大道理，黄渤讲故事没什么技巧，如果不是考虑他是新导演我会给4分,0
2,真心難看 不知道在看什麼電影 真心後悔自己來看 拍了三年一個爛戲 ，現在真的是個有名的人 都...,0
3,王渤与王宝强这次让人有失望,0
4,不好看。不好看。不好看。重要的事情说三遍…,0


### 积极评论

In [5]:
pos.head()

,0,1
0,19岁生日第一次和和家里人看了电影,1
1,1分钱坐公交车（每天两次，活动期限11月28日至1月27日，之后至三月底每周一是一分钱）,1
2,2019年7月才把主题曲听懂，可惜太痛了,1
3,38.7万人评分9.0，除了自来水。没可能进行。这故事有点达拉斯买家俱乐部。但是坏制度下的好...,1
4,按耐住这些天被朋友圈剧透的风险，终于今日亲自观看了这部良心电影！许多时候艺术来源于生活，电影...,1


## 5.数据连接

In [6]:
comment=np.concatenate((pos[0], neg[0]))
y = np.concatenate((np.ones(len(pos),dtype=int), np.zeros(len(neg),dtype=int)))
comment = comment.astype(str).tolist()

In [7]:
comment[:10]

['19岁生日第一次和和家里人看了电影',
 '1分钱坐公交车（每天两次，活动期限11月28日至1月27日，之后至三月底每周一是一分钱）',
 '2019年7月才把主题曲听懂，可惜太痛了',
 '38.7万人评分9.0，除了自来水。没可能进行。这故事有点达拉斯买家俱乐部。但是坏制度下的好人，还得为主旋律服务。难为文牧野，徐峥，宁浩了。这是一部探底的现实主义题材，如果什么都不让拍，怎么反腐，如何医改。所以都在套路中。徐峥是好演员，而且这片接地气。拿钱买命，他们违法么，有罪么，如果不牟利呢，如果赔钱呢？这钱该谁贴呢？领导。我就问你，为什么一定要死人，才能促进一个制度的改变呢？为什么要这么大代价？在一个国度里，想有尊严的活着，哪怕体面的死去，难道都这么难么？面对钱，谁没欲望。那个不被诱惑。但是，面对病，大家都是平等的，只是你用什么方式续命。我大概理解很多评分高的原因，老百姓想看什么。',
 '按耐住这些天被朋友圈剧透的风险，终于今日亲自观看了这部良心电影！许多时候艺术来源于生活，电影亦如此，越贴近生活，越能反映普罗大众的心声，就越能得到群众的拥戴，此电影能火，靠的不是名导小鲜肉，而是为民请命，替弱势群体伸张，唤醒良知！好的电影，不应该只是搏众人一笑，只宣场真善美，还应能敢于揭露社会的阴暗面，我们需要的就是这种有内涵有灵魂能触动到心灵深处的良心电影！\n商业电影，追求永远是票房，但此次的大热，实在是理所当然，因为它撕开了和谐社会的一个口，能敢于说事实，敢于揭露社会阴暗面，对于此良心电影，我尽一点绵力，为它打CALL，宣传，只希望中国电影能越来越多此类的好电影！',
 '把看病贵体现的玲离尽致，现实中要是没有保险，真的是病不起，就连死都死不起。',
 '把人的在遇到疾病时的那种无奈，无助，对求生的渴望，拍的淋漓尽致',
 '把手握紧，你什么也没有。把手张开，你拥有的是一切。',
 '把现实拍成电影，堪称完美的情节。有搞笑情节，也有感人情节，支持多上映这种类型的电影',
 '把中国曾经的矛盾表现了出来，很真实']

## 6.分词

In [8]:
def chinese_word_cut(text):
    """
    分词
    """
    text = [" ".join(jieba.cut(document)) for document in text]
    return text

In [9]:
comment =chinese_word_cut(comment)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.148 seconds.
Prefix dict has been built succesfully.


### 分词结果

In [10]:
comment[:10]

['19 岁 生日 第一次 和 和 家里人 看 了 电影',
 '1 分钱 坐 公交车 （ 每天 两次 ， 活动 期限 11 月 28 日至 1 月 27 日 ， 之后 至 三月底 每周 一是 一分钱 ）',
 '2019 年 7 月 才 把 主题曲 听懂 ， 可惜 太痛 了',
 '38.7 万人 评分 9.0 ， 除了 自来水 。 没 可能 进行 。 这 故事 有点 达拉斯 买家 俱乐部 。 但是 坏 制度 下 的 好人 ， 还 得 为 主旋律 服务 。 难为 文 牧野 ， 徐峥 ， 宁浩 了 。 这是 一部 探底 的 现实主义 题材 ， 如果 什么 都 不让 拍 ， 怎么 反腐 ， 如何 医改 。 所以 都 在 套路 中 。 徐峥 是 好 演员 ， 而且 这片 接地 气 。 拿 钱 买命 ， 他们 违法 么 ， 有罪 么 ， 如果 不 牟利 呢 ， 如果 赔钱 呢 ？ 这钱 该 谁 贴 呢 ？ 领导 。 我 就 问 你 ， 为什么 一定 要死 人 ， 才能 促进 一个 制度 的 改变 呢 ？ 为什么 要 这么 大 代价 ？ 在 一个 国度 里 ， 想 有 尊严 的 活着 ， 哪怕 体面 的 死去 ， 难道 都 这么 难 么 ？ 面对 钱 ， 谁 没 欲望 。 那个 不 被 诱惑 。 但是 ， 面对 病 ， 大家 都 是 平等 的 ， 只是 你 用 什么 方式 续命 。 我 大概 理解 很多 评分 高 的 原因 ， 老百姓 想 看 什么 。',
 '按 耐住 这些 天 被 朋友圈 剧透 的 风险 ， 终于 今日 亲自 观看 了 这部 良心 电影 ！ 许多 时候 艺术 来源于 生活 ， 电影 亦 如此 ， 越 贴近生活 ， 越能 反映 普罗 大众 的 心声 ， 就 越 能 得到 群众 的 拥戴 ， 此 电影 能火 ， 靠 的 不是 名导 小 鲜肉 ， 而是 为民请命 ， 替 弱势群体 伸张 ， 唤醒 良知 ！ 好 的 电影 ， 不 应该 只是 搏 众人 一笑 ， 只 宣场 真善美 ， 还 应能 敢于 揭露 社会 的 阴暗面 ， 我们 需要 的 就是 这种 有 内涵 有 灵魂 能 触动 到 心灵深处 的 良心 电影 ！ \n 商业 电影 ， 追求 永远 是 票房 ， 但 此次 的 大热 ， 实在 是 理所当然 ， 因为 它 撕开 了 和谐社会 的 一个 口 ， 能 

## 7.去停用词

In [11]:

def get_stopword_list():
    """
    获取停用词列表
    """
    stop_word_path = '../../data/day10-nlp-data/chineseStopWords.txt'
    stopword_list = [sw.replace('\n', '') for sw in open(stop_word_path, encoding='gb18030').readlines()]
    return stopword_list
stopword_list = get_stopword_list()

In [12]:
def remove_stopwords(text):
    """
    去停用词
    """
    mytext = ''
    text= text.split()
    for i,j in enumerate(text):  
        if j not in stopword_list:
            mytext+=j
            if i!=len(text)-1:
                mytext+=' '
    return mytext

In [13]:
comment = [remove_stopwords(text) for text in comment]

### 去停用词结果

In [14]:
comment[:10]

['19 岁 生日 第一次 家里人 ',
 '1 分钱 坐 公交车 两次 活动 期限 11 月 28 日至 1 月 27 日 三月底 每周 一是 一分钱 ',
 '2019 7 月 主题曲 听懂 可惜 太痛 ',
 '38.7 万人 评分 9.0 自来水 没 故事 达拉斯 买家 俱乐部 坏 制度 好人 主旋律 服务 难为 文 牧野 徐峥 宁浩 这是 一部 探底 现实主义 题材 不让 拍 反腐 医改 套路 徐峥 好 演员 这片 接地 气 钱 买命 违法 有罪 牟利 赔钱 这钱 贴 领导 问 要死 才能 制度 改变 代价 国度 想 尊严 活着 体面 死去 难道 难 面对 钱 没 欲望 诱惑 面对 病 平等 方式 续命 大概 理解 很多 评分 高 原因 老百姓 想 ',
 '耐住 天 朋友圈 剧透 风险 终于 今日 亲自 观看 良心 艺术 来源于 生活 亦 越 贴近生活 越能 普罗 大众 心声 越 群众 拥戴 能火 名导 鲜肉 为民请命 弱势群体 伸张 唤醒 良知 好 搏 众人 一笑 宣场 真善美 应能 敢于 揭露 社会 阴暗面 内涵 灵魂 触动 心灵深处 良心 商业 追求 永远 票房 此次 大热 实在 理所当然 撕开 和谐社会 口 敢于 事实 敢于 揭露 社会 阴暗面 良心 绵力 CALL 宣传 希望 中国 越来越 此类 好 ',
 '看病贵 体现 玲离 尽致 现实 保险 病不起 死 死 不起 ',
 '疾病 那种 无奈 无助 求生 渴望 拍 淋漓尽致',
 '把手 握紧 把手 张开 拥有 ',
 '现实 拍成电影 堪称 完美 情节 搞笑 情节 感人 情节 支持 上映 类型 ',
 '中国 矛盾 表现 真实']

## 8.加载词向量模型

In [1]:
model = Word2Vec.load('model/Word60.model') 

NameError: name 'Word2Vec' is not defined

In [ ]:

#创建词典，并将词语与向量相对应
def create_dictionaries(model=None,comment=None):
    """
    建立词语-向量映射表,
    model:词向量模型
    comment:预处理后的数据集
    
    返回值：
    w2indx 词典（格式为：索引+词语）
    w2vec  词语对应的向量
    comment  向量化后的数据集（词语-词向量）

    """
    if (comment is not None) and (model is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.wv.vocab.keys(),
                            allow_update=True)
        w2indx = {v: k+1 for k, v in gensim_dict.items()}#所有频数超过10的词语的索引
        w2vec = {word: model[word] for word in w2indx.keys()}#所有频数超过10的词语的词向量
        def parse_dataset(comment):
            ''' Words become integers
            '''
            data=[]
            for sentence in comment:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2vec[word])
                    except:
                        a = np.empty(60)
                        new_txt.append(a)
                data.append(new_txt)
            return data
        comment=parse_dataset(comment)
        #每个句子所含词语对应的索引，所以句子中含有频数小于1的词语，索引为0
        comment= sequence.pad_sequences(comment, maxlen=50)
        return w2indx, w2vec,comment
    else:
        print('No data provided...')

In [ ]:
_,_,comment = create_dictionaries(model,comment)

## 9.word2vec取平均值

In [ ]:
def word2vec_to_one(comment=None):
    """
     将一句话中每个词语不同的维度计算平均值，得到最终一个向量作为特征
    """
    com_vec = []
    for sentence in comment:
        list_ = []  
        for i in range(60):
            sum_ = 0
            for j in sentence:
                sum_+=j[i]
            avg = sum_/len(sentence)
            list_.append(avg)
        com_vec.append(list_)
    return np.array(com_vec)

In [ ]:
X = word2vec_to_one(comment)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state = 1)
print(x_train.shape,y_train.shape)

## 10.创建模型

In [ ]:
"""
1.初始化：
    glorot_uniform均匀分布初始化
    优化函数Adam
"""

init = K.initializers.glorot_uniform(seed=2)
simple_adam = K.optimizers.Adam()

"""
2.定义DNN模型：
    搭建的DNN由输入层、隐藏层、输出层和sigmoid函数组成，其中输入层由训练集的维度决定，隐藏层有一层，
    包括32个神经元，之后就是输出层，由1个神经元组成，最后，sigmoid，用于解决二分类问题而创建。
"""
model = K.models.Sequential()
model.add(K.layers.Dense(units=32, input_dim=x_train.shape[1], kernel_initializer=init, activation='sigmoid'))
model.add(K.layers.Dense(units=1, kernel_initializer=init, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=simple_adam, metrics=['accuracy'])

"""
3.训练模型fit
"""
h = model.fit(X_new_train, y_train, batch_size=100, epochs=5, shuffle=True, verbose=1)

"""
4.检验模型evaluate
"""
eval = model.evaluate(X_test, y_test, verbose=0)
print("Evaluation on test data: loss = %0.6f accuracy = %0.2f%% \n" \
          % (eval[0], eval[1] * 100))